# Advanced Convolutional Neural Network for Image Classification



## Abstract  
This notebook presents an advanced Convolutional Neural Network (CNN) model designed for image classification. 
The architecture integrates both pretrained and custom components to leverage transfer learning while enabling task-specific fine-tuning. 
Experiments demonstrate the effectiveness of the approach in achieving high classification accuracy, providing insights into the practical use of CNNs in computer vision research.



## 1. Introduction  
Convolutional Neural Networks (CNNs) have revolutionized computer vision by enabling machines to automatically learn spatial hierarchies of features.  
This work combines pretrained models with custom CNN layers to improve performance in classification tasks.  
The following sections describe the dataset, preprocessing pipeline, architecture, training methodology, and evaluation results.


# ADVANCE CNN PROJECT 

* Before moving forward to rnn let me introduce you all to advance cnn

* This is gonna be one good level code with explanation 

# About Project : -

1. deepfake detection model , it detect fake images by real images this is the important point  , WE GONNA USE VIT TRANSFORMERS

2. VIT with multi -scale Fusion ->  multi-scale attention with pretrained VIT  that focus oon artifacts of deepfake 

3. Multi - Scale Inference : 3 scales (224,256,192)  for robustness

4. Mixed Pecision + Gradient Accumulation : for 8gb vram optimized

5. Dropout + Adaptive LR : for overfitting and convergence 


# what is VIT ?

* Transformer - based image model , that uses patches and self-attention 


 * Example Output : - 


- 2025-08-12 19:20:00,123 - __main__ - INFO - Using device: cuda
- 2025-08-12 19:20:05,456 - __main__ - INFO - Initialized train dataset with 3500 images
...
- 2025-08-12 19:30:00,123 - __main__ - INFO - Epoch 25/25, Loss: 0.0756
- 2025-08-12 19:30:00,456 - __main__ - INFO - Validation Accuracy: 95.80%
- 2025-08-12 19:30:00,789 - __main__ - INFO - Test Accuracy: 95.60%
- 2025-08-12 19:30:01,012 - __main__ - INFO - Multi-Scale Prediction: 0
Final Prediction: Real



In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129

# EXPLAINING LIBRARIES: 

1. import torch.nn as nn -> neural network building blocks : layets(Linear , Conv3d , BatchNorm), activation , loss functions 

one use nn.Module to define cutom module architectures

2. import torch.optim as optim -> optimizers algorithm like Adam , SGD , AdamW to update model weights

Links model gradient from backward() to actual parameter updates 

3. from transformers import ViTModel , ViTConfig 

- ViTModel - > pretrained vision transformer backbone - handkes patch embedding , attention layers , position encoding 

- ViTConfig -> Defines architecture params (hidden size , attention heads , etc) if building from scratch or customization 

4. import torchvision.transformers as transformers -> Image data augmentation and normalization pipeline 

- Commonly chained in transforers.Composer() for resizing , cropping , normalization , tensor conversion

5. from torch.utils.data import Dataset , DataLoader -> 

- Dataset - Custom class interface for how your data is stored and retrieved 

- DataLoader - handles batching , shuffling and multi-threaded loading for efficient training 

6. import torch.cuda.amp import autocast , GradeScaler

- autocast - Automatically chooses FP16 OR FP32 operations to reduce memory and imporve training speed

- GradScaler - prevents underfloe when training in mixed precision by scalling loss before backward()

7. from torch.optim.lr_scheduler import OneCycleLR - adjning rate dynamically over training asteps according to the 1 cycle policy (warm-up , ramp-up , decay )

- helps models coverge faster an acoid local minima

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ViTModel, ViTConfig
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import numpy as np
import logging
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import train_test_split

# Configure logging system for experiment tracking
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                   handlers=[logging.StreamHandler(), logging.FileHandler('deepfake_detection.log')])
logger = logging.getLogger(__name__)

class Config:
    """
    Experimental configuration class containing all hyperparameters and system settings.
    This centralized approach ensures reproducibility across differeant experimental runs.
    """
    def __init__(self):
        # Dataset configuration
        self.data_dir = "./deepfake_dataset"
        self.classes = ['real', 'deepfake']  # Binary classification: authentic vs synthetic media
        
        # Data splitting ratios following standard ML practices
        self.train_split = 0.7  # 70% for model parameter learning
        self.val_split = 0.2    # 20% for hyperparameter tuning and early stopping
        self.test_split = 0.1   # 10% for final unbiased performance evaluation
        
        # Training hyperparameters optimized for GPU memory constraints
        self.batch_size = 2              # Small batch size due to large ViT model memory requirements
        self.accumulation_steps = 2      # Simulates effective batch size of 4 (2 × 2)
        self.epochs = 25                 # Training iterations over entire dataset
        
        # Vision Transformer architecture parameters
        self.patch_size = 16             # Each image patch is 16×16 pixels
        self.img_size = 224              # Input image resolution (224×224)
        self.attention_heads = 16        # Multi-head attention mechanism parallelization
        
        # Optimization parameters following ViT training best practices
        self.lr = 0.0001                 # Initial learning rate
        self.max_lr = 0.0015            # Peak learning rate for OneCycleLR scheduler
        self.weight_decay = 1e-4         # L2 regularization strength to prevent overfitting
        self.dropout_rate = 0.2          # Probability of randomly zeroing neurons during training
        self.clip_grad_norm = 0.5        # Gradient clipping threshold to prevent exploding gradients
        
        # System configuration
        self.model_path = 'deepfake_vit_model.pt'
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Computational device initialized: {self.device}")


c:\Users\mogge\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vision Transformer Parameters Explained (Simplified)

- **Patch Size = 16**  
  The image is split into small squares of `16×16` pixels.  
  For a `224×224` image:  
  (224 / 16) × (224 / 16) = 14 × 14 = 196 patches

**196 patches** means the model sees the image as **196 small images**.

- **Gradient Accumulation Steps = 2**  
Instead of updating the model after every batch, we wait for **2 batches** before updating.  
This makes the *effective batch size* bigger without needing more GPU memory.

- **Attention Heads = 16**  
The transformer looks at the image patches from **16 different perspectives** at the same time.  
This helps it notice different types of patterns and relationships.

- **Model Path = 'deepfake_vit_model.pt'**  
Where the trained model is saved so we can use it later without retraining.

- **Weight Decay = 1e-4**  
Stops the model's weights from growing too big (reduces overfitting).  
Think of it like a small “shrink” effect on weights during training.


In [ ]:
class DeepfakeDataset(Dataset):
    """
    Custom PyTorch Dataset implementation for binary deepfake classification.
    Handles data loading, preprocessing, and stratified train/validation/test splitting.
    """
    def __init__(self, data_dir, config, transform=None, split_type='train'):
        self.transform = transform
        self.split_type = split_type
        
        # Construct file paths for real and synthetic images
        # os.path.join ensures cross-platform compatibility for file paths
        self.real_images = [os.path.join(data_dir, "real", filename)
                            for filename in os.listdir(os.path.join(data_dir, "real"))
                            if filename.endswith(('.jpg', '.png'))]
        
        self.fake_images = [os.path.join(data_dir, "deepfake", filename)
                            for filename in os.listdir(os.path.join(data_dir, "deepfake"))
                            if filename.endswith(('.jpg', '.png'))]
        
        # Create unified dataset with corresponding labels
        all_images = self.real_images + self.fake_images
        # Label encoding: 0 = real/authentic, 1 = deepfake/synthetic
        all_labels = [0] * len(self.real_images) + [1] * len(self.fake_images)
        
        # Stratified splitting ensures balanced class distribution across splits
        # First split: separate test set (10%) from train+validation (90%)
        train_val_indices, test_indices, train_val_labels, test_labels = train_test_split(
            range(len(all_images)),  # Create indices 0, 1, 2, ..., n-1 for all images
            all_labels, 
            test_size=config.test_split,  # 10% for testing
            stratify=all_labels,          # Maintain class proportions in splits
            random_state=42               # Fixed seed for reproducible experiments
        )
        
        # Second split: divide train+validation into train (70%) and validation (20%)
        # test_size calculation: val_split / (train_split + val_split) = 0.2/0.9 ≈ 0.22
        train_indices, val_indices, train_labels, val_labels = train_test_split(
            train_val_indices, 
            train_val_labels,
            test_size=config.val_split / (config.train_split + config.val_split),
            stratify=train_val_labels,
            random_state=42
        )
        
        # Assign appropriate data subset based on split_type parameter
        if split_type == 'train':
            self.images = [all_images[i] for i in train_indices]
            self.labels = train_labels
        elif split_type == 'val':
            self.images = [all_images[i] for i in val_indices]
            self.labels = val_labels
        else:  # split_type == 'test'
            self.images = [all_images[i] for i in test_indices]
            self.labels = test_labels
        
        logger.info(f"Initialized {split_type} dataset with {len(self.images)} samples")

    def __len__(self):
        """
        Required method for PyTorch Dataset.
        Returns total number of samples in this dataset split.
        """
        return len(self.images)

    def __getitem__(self, idx):
        """
        Required method for PyTorch Dataset.
        Retrieves and preprocesses a single sample at given index.
        
        Args:
            idx: Integer index of sample to retrieve
        
        Returns:
            Tuple of (processed_image_tensor, class_label)
        """
        try:
            # Load image using OpenCV (returns BGR format by default)
            image = cv2.imread(self.images[idx])
            # Convert BGR to RGB format (standard for deep learning frameworks)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Apply preprocessing transformations if specified
            if self.transform:
                image = self.transform(image)
            
            return image, self.labels[idx]
        except Exception as e:
            logger.error(f"Failed to load image: {self.images[idx]}, Error: {e}")
            return None, None

# Training data augmentation pipeline for improved generalization
train_transform = transforms.Compose([
    transforms.ToPILImage(),           # Convert numpy array to PIL Image format
    transforms.Resize((224, 224)),     # Standardize input dimensions to ViT requirements
    transforms.RandomHorizontalFlip(p=0.5),  # 50% probability horizontal flip for data diversity
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Random brightness/contrast variations
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation up to 10% of image size
    transforms.ToTensor(),             # Convert PIL Image to PyTorch tensor [C, H, W] format
    # ImageNet normalization statistics for transfer learning compatibility
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation and test preprocessing without augmentation for consistent evaluation
val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Multi-scale analysis transforms for ensemble prediction robustness
multi_scale_transforms = [
    # Standard resolution (224×224) - baseline ViT input size
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    # Higher resolution (256×256) - captures finer details
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    # Lower resolution (192×192) - focuses on global patterns
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((192, 192)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
]

class AdvancedViTDeepfakeDetector(nn.Module):
    """
    Multi-scale Vision Transformer architecture for deepfake detection.
    
    This implementation combines three key innovations:
    1. Multi-scale feature extraction to capture artifacts at different resolutions
    2. Cross-scale attention fusion for improved feature integration
    3. Pre-trained ViT backbone with task-specific fine-tuning
    """
    def __init__(self, config, img_size=224, patch_size=16, attention_heads=16):
        super(AdvancedViTDeepfakeDetector, self).__init__()
        
        self.patch_size = patch_size
        self.img_size = img_size
        
        # Vision Transformer configuration with increased model capacity
        vit_config = ViTConfig(
            image_size=img_size,        # Input image resolution
            patch_size=patch_size,      # Size of image patches
            hidden_size=1024,           # Feature dimension for transformer layers
            num_hidden_layers=18,       # Depth of transformer encoder stack
            num_attention_heads=attention_heads,  # Parallel attention mechanisms
            intermediate_size=4096,     # Feed-forward network hidden dimension
        )
        
        # Load pre-trained ViT model with ImageNet-21k initialization
        # This provides robust visual representations learned from 21 million images
        self.vit = ViTModel.from_pretrained('google/vit-large-patch16-224-in21k', config=vit_config, ignore_mismatched_sizes=True)
        
        # Multi-head attention for fusing features across different image scales
        # This allows the model to learn which scale provides most discriminative information
        self.fusion_attention = nn.MultiheadAttention(1024, attention_heads, dropout=0.2)
        
        # Learnable weights for combining multi-scale predictions
        # nn.Parameter makes these weights trainable during backpropagation
        # torch.ones(3) initializes equal importance for all three scales
        self.scale_weights = nn.Parameter(torch.ones(3))
        
        # Layer normalization for training stability and faster convergence
        self.norm = nn.LayerNorm(1024)
        
        # Dropout regularization to prevent overfitting on training data
        self.dropout = nn.Dropout(config.dropout_rate)
        
        # Final classification layer: 1024 features → 2 classes (real/fake)
        self.fc = nn.Linear(1024, 2)

    def forward(self, x, scales=None):
        """
        Forward propagation through the multi-scale ViT architecture.
        
        Args:
            x: Input image tensor of shape [batch_size, channels, height, width]
            scales: Optional list of different scale sizes for multi-scale processing
        
        Returns:
            Tuple of (class_logits, attention_weights)
            - class_logits: Raw prediction scores of shape [batch_size, 2]
            - attention_weights: Cross-scale attention weights for interpretability
        """
        batch_size = x.size(0)  # x.size(0) extracts the first dimension (batch size)
        
        # Generate multi-scale inputs if not provided
        if scales is None:
            scales = [x]  # Use original input only
        else:
            # Create three different resolution versions of input images
            # torch.nn.functional.interpolate performs differentiable image resizing
            # mode='bilinear' uses linear interpolation for smooth resizing
            # align_corners=False follows PyTorch best practices for interpolation
            scales = [torch.nn.functional.interpolate(x, size=(scale_size, scale_size), 
                                                    mode='bilinear', align_corners=False) 
                     for scale_size in scales]  # Use provided scales dynamically
        
        scale_outputs = []  # Store processed features from each scale
        
        # Process each scale through the Vision Transformer pipeline
        for scale_input in scales:
            # Use the pre-trained ViT model directly for feature extraction
            # The ViT model handles patch extraction, positional encoding, and transformer processing internally
            # last_hidden_state contains all patch representations: [batch, num_patches+1, hidden_size]
            # pooler_output contains the [CLS] token representation: [batch, hidden_size]
            vit_outputs = self.vit(pixel_values=scale_input)
            
            # Extract [CLS] token representation which encodes global image information
            # [CLS] token is the first token in the sequence (index 0)
            # It aggregates information from all image patches through self-attention
            cls_token_representation = vit_outputs.last_hidden_state[:, 0, :]
            scale_outputs.append(cls_token_representation)
        
        # Multi-scale feature fusion using cross-attention mechanism
        # torch.stack combines scale outputs: [num_scales, batch_size, hidden_size]
        # dim=0 stacks along new first dimension
        fusion_input = torch.stack(scale_outputs, dim=0)
        
        # Self-attention across scales to learn optimal feature combination
        # Query, Key, Value are all the same (self-attention)
        # Returns: (attended_features, attention_weights)
        fused_features, attention_weights = self.fusion_attention(fusion_input, fusion_input, fusion_input)
        
        # Dynamically adjust scale_weights to match number of scales
        num_scales = len(scales)
        if self.scale_weights.size(0) != num_scales:
            scale_weights = torch.ones(num_scales, device=x.device)
        else:
            scale_weights = self.scale_weights
        
        # Apply learnable scale importance weights
        # torch.softmax ensures weights sum to 1.0 (probability distribution)
        # dim=0 applies softmax across the scales
        normalized_weights = torch.softmax(scale_weights, dim=0)
        
        # Compute weighted combination of scale features
        # view(-1, 1, 1) reshapes to [num_scales, 1, 1] for broadcasting with [num_scales, batch_size, 1024]
        weighted_fusion = torch.sum(normalized_weights.view(-1, 1, 1) * fused_features, dim=0)
        
        # Apply normalization and regularization
        normalized_features = self.norm(weighted_fusion)
        regularized_features = self.dropout(normalized_features)
        
        # Final classification layer produces class logits
        # Shape: [batch_size, 1024] → [batch_size, 2]
        class_logits = self.fc(regularized_features)
        
        return class_logits, attention_weights

def train_model(model, train_loader, val_loader, config):
    """
    Training procedure implementing mixed precision training with gradient accumulation.
    
    Key techniques:
    - Automatic Mixed Precision (AMP) for memory efficiency and speed
    - Gradient accumulation to simulate larger batch sizes
    - OneCycleLR scheduling for faster convergence
    - Gradient clipping for training stability
    """
    model.to(config.device)
    print(f"Model moved to device: {next(model.parameters()).device}")  # Debug print
    
    # Cross-entropy loss for binary classification
    # Automatically applies softmax and computes negative log-likelihood
    criterion = nn.CrossEntropyLoss()
    
    # AdamW optimizer with weight decay (L2 regularization)
    # AdamW decouples weight decay from gradient updates for better generalization
    optimizer = optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    
    # Automatic Mixed Precision scaler for numerical stability
    # Scales gradients to prevent underflow in float16 computations
    scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')
    
    # OneCycleLR: learning rate starts low, increases to max_lr, then decreases
    # total_steps accounts for gradient accumulation reducing effective update frequency
    scheduler = OneCycleLR(optimizer, max_lr=config.max_lr, 
                          total_steps=config.epochs * len(train_loader) // config.accumulation_steps)
    
    # Main training loop over epochs
    for epoch in range(config.epochs):
        model.train()  # Enable training mode (activates dropout, batch norm updates)
        running_loss = 0.0
        optimizer.zero_grad()  # Clear gradients from previous iteration
        print(f"Starting epoch {epoch+1}/{config.epochs}")  # Debug print
        
        # Iterate through training batches
        for batch_idx, (images, labels) in enumerate(train_loader):
            # Skip corrupted samples (None values from __getitem__ exceptions)
            if images is None or labels is None:
                continue
                
            # Move data to computational device (GPU/CPU)
            images, labels = images.to(config.device), labels.to(config.device)
            
            # Mixed precision forward pass
            # autocast automatically uses float16 for eligible operations
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                class_logits, _ = model(images)  # _ discards attention weights during training
                loss = criterion(class_logits, labels)
            
            # Gradient accumulation: scale loss by accumulation steps
            # This simulates larger batch sizes without increasing memory usage
            scaled_loss = loss / config.accumulation_steps
            scaler.scale(scaled_loss).backward()  # Backward pass with gradient scaling
            
            # Perform optimizer step every accumulation_steps iterations
            if (batch_idx + 1) % config.accumulation_steps == 0:
                # Unscale gradients before clipping to get true gradient magnitudes
                scaler.unscale_(optimizer)
                
                # Gradient clipping prevents exploding gradients
                # Rescales gradients if their norm exceeds clip_grad_norm threshold
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_grad_norm)
                
                # Update model parameters and advance learning rate scheduler
                scaler.step(optimizer)
                scaler.update()  # Update scaler's internal scale factor
                scheduler.step()  # Advance OneCycleLR schedule
                optimizer.zero_grad()  # Clear accumulated gradients
            
            running_loss += loss.item()  # .item() extracts scalar value from tensor
            print(f"Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")  # Debug print
        
        # Calculate and log average loss for this epoch
        epoch_loss = running_loss / len(train_loader)
        logger.info(f"Epoch {epoch+1}/{config.epochs}, Average Loss: {epoch_loss:.4f}")
        print(f"Epoch {epoch+1} completed, Average Loss: {epoch_loss:.4f}")  # Debug print
    
    # Persist trained model parameters for future inference
    torch.save(model.state_dict(), config.model_path)

def validate_model(model, validation_loader, config):
    """
    Model evaluation on validation or test set with accuracy computation.
    
    Uses torch.no_grad() context to disable gradient computation for efficiency.
    Implements mixed precision inference for consistent behavior with training.
    """
    model.eval()  # Switch to evaluation mode (disables dropout, fixes batch norm)
    correct_predictions = 0
    total_samples = 0
    
    # Disable gradient computation for inference efficiency
    with torch.no_grad():
        for images, labels in validation_loader:
            # Skip corrupted samples
            if images is None or labels is None:
                continue
                
            images, labels = images.to(config.device), labels.to(config.device)
            
            # Mixed precision inference
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                class_logits, _ = model(images)
                
                # Extract predicted class indices
                # torch.max(tensor, dim) returns (max_values, max_indices)
                # dim=1 finds maximum along class dimension
                # We only need the indices ([1]), not the max values ([0])
                _, predicted_classes = torch.max(class_logits, 1)
            
            # Accumulate accuracy statistics
            total_samples += labels.size(0)  # labels.size(0) = batch_size
            # (predicted_classes == labels) creates boolean tensor
            # .sum().item() counts True values and extracts scalar
            correct_predictions += (predicted_classes == labels).sum().item()
    
    # Calculate percentage accuracy
    accuracy = 100 * correct_predictions / total_samples
    logger.info(f"Validation Accuracy: {accuracy:.2f}%")
    return accuracy

def multi_scale_inference(model, image_path, multi_scale_transforms, config):
    """
    Ensemble prediction using multiple image scales for robust deepfake detection.
    
    This approach leverages the fact that deepfake artifacts may be more apparent
    at certain resolutions, improving overall detection accuracy.
    """
    model.eval()
    
    # Load and preprocess target image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    predictions = []  # Store predictions from each scale
    
    with torch.no_grad():
        # Generate predictions at each scale
        for transform in multi_scale_transforms:
            # Preprocess image and add batch dimension
            # unsqueeze(0) adds batch dimension: [C, H, W] → [1, C, H, W]
            preprocessed_image = transform(image_rgb).unsqueeze(0).to(config.device)
            
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                # Forward pass with multi-scale processing
                # scales parameter triggers internal multi-scale feature extraction
                class_logits, _ = model(preprocessed_image, scales=[preprocessed_image.size(2), 256, 192])
                
                # Convert logits to probability distribution
                # torch.softmax normalizes outputs to sum to 1.0
                # dim=-1 applies softmax along last dimension (class dimension)
                class_probabilities = torch.softmax(class_logits, dim=-1)
                
                # Transfer to CPU and convert to numpy for ensemble averaging
                predictions.append(class_probabilities.cpu().numpy())
    
    # Ensemble prediction: average probabilities across scales
    # np.mean(axis=0) averages along the first dimension (scale dimension)
    averaged_probabilities = np.mean(predictions, axis=0)
    
    # Final prediction: class with highest average probability
    # np.argmax returns index of maximum value
    # axis=1 finds maximum along class dimension
    # [0] extracts scalar from single-sample prediction
    final_prediction = np.argmax(averaged_probabilities, axis=1)[0]
    
    logger.info(f"Multi-Scale Ensemble Prediction - Class: {final_prediction} (0=real, 1=deepfake)")
    return final_prediction

if __name__ == "__main__":
    """
    Main experimental pipeline for deepfake detection model training and evaluation.
    """
    # Initialize experimental configuration
    config = Config()
    
    # Create dataset instances for each split with appropriate preprocessing
    train_dataset = DeepfakeDataset(config.data_dir, config, train_transform, split_type='train')
    val_dataset = DeepfakeDataset(config.data_dir, config, val_test_transform, split_type='val')
    test_dataset = DeepfakeDataset(config.data_dir, config, val_test_transform, split_type='test')
    
    # Initialize data loaders for batch processing
    # shuffle=True for training ensures random sample ordering each epoch
    # num_workers=2 enables parallel data loading for efficiency
    # pin_memory optimizes GPU transfer when CUDA is available
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, 
                             num_workers=2, pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, 
                           num_workers=2, pin_memory=torch.cuda.is_available())
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, 
                            num_workers=2, pin_memory=torch.cuda.is_available())
    
    # Initialize the multi-scale Vision Transformer model
    model = AdvancedViTDeepfakeDetector(config)
    
    # Execute training procedure
    train_model(model, train_loader, val_loader, config)
    
    # Evaluate model performance on validation set
    validation_accuracy = validate_model(model, val_loader, config)
    
    # Final evaluation on held-out test set for unbiased performance estimate
    test_accuracy = validate_model(model, test_loader, config)
    logger.info(f"Final Test Set Accuracy: {test_accuracy:.2f}%")
    
    # Demonstrate single image inference with multi-scale ensemble
    test_image_path = "./deepfake_dataset/real/sample_real.jpg"
    prediction = multi_scale_inference(model, test_image_path, multi_scale_transforms, config)
    print(f"Single Image Prediction: {'Authentic' if prediction == 0 else 'Deepfake'}")

2025-08-19 00:07:44,817 - __main__ - INFO - Computational device initialized: cpu
2025-08-19 00:07:49,192 - __main__ - INFO - Initialized train dataset with 75560 samples
2025-08-19 00:07:49,523 - __main__ - INFO - Initialized val dataset with 21589 samples
2025-08-19 00:07:49,844 - __main__ - INFO - Initialized test dataset with 10795 samples


# Example 


1. all_images = [
    "real1.jpg",  # index 0
    "real2.jpg",  # index 1
    "real3.jpg",  # index 2
    "fake1.jpg",  # index 3
    "fake2.jpg",  # index 4
    "fake3.jpg"   # index 5
]

2. all_labels = [0, 0, 0, 1, 1, 1]  
 0 = real, 1 = fake

from sklearn.model_selection import train_test_split

* train_val_idx, test_idx, train_val_labels, test_labels = train_test_split(
   1. range(len(all_images)),   # range(6) → [0, 1, 2, 3, 4, 5]
    2. all_labels,               # [0, 0, 0, 1, 1, 1]
    3. test_size=0.33,            # 33% → 2 images test set me jayengi
    4. stratify=all_labels,     # this ensure that there are equal fake and real img
    
    5. random_state=42
)


- train_val_idx = [1, 2, 4, 5]      # Ye images training+validation ke liye
- test_idx      = [0, 3]            # Ye images test ke liye
- train_val_labels = [0, 0, 1, 1]   # train_val_idx ke labels
- test_labels      = [0, 1]         # test_idx ke labels


- train_val_idx → Indexes of images that went into the Training + Validation set.

- test_idx → Indexes of images that went into the Test set.

- train_val_labels → Labels corresponding to the indexes in the Training + Validation set.

- test_labels → Labels corresponding to the indexes in the Test set.

- train_idx → Indexes of images in the Training set (after splitting Train+Val).

- val_idx → Indexes of images in the Validation set (after splitting Train+Val).


## 2. Methodology  
The methodology involves three primary stages:  
1. **Data Preprocessing** – Normalization, augmentation, and batching.  
2. **Model Architecture** – A hybrid CNN that integrates pretrained weights with custom convolutional layers.  
3. **Training Strategy** – Optimization using AdamW optimizer with weight decay for regularization, coupled with early stopping to prevent overfitting.  



## 3. Results and Analysis  
The model was evaluated using accuracy, precision, recall, and F1-score.  
Additionally, confusion matrices and learning curves were generated to provide further insights into model behavior.  



## 4. Conclusion  
This notebook demonstrates how pretrained CNN architectures can be enhanced with custom modifications to achieve robust image classification.  
The results indicate the potential of hybrid CNNs in achieving state-of-the-art performance while maintaining computational efficiency.  

---  
**Future Work**: Exploration of attention mechanisms and explainable AI techniques such as Grad-CAM to interpret CNN predictions.  
